In [ ]:
#!conda info
#!conda list

In [ ]:
import requests
import json
import sqlalchemy
from IPython.display import JSON
from sqlalchemy import Table, Column, Integer, Float, String, MetaData, DateTime, create_engine
from sqlalchemy.orm import sessionmaker
import datetime
import mysql.connector
import config #make sure this matches the name of your config file
from config import user, passw, uri# from config import user, passw, uri
 

In [ ]:
engine = create_engine(f"mysql+mysqlconnector://{config.user}:{config.passw}@{config.uri}:3306/wheelieGood", echo=True)
print(engine.url)

In [ ]:
meta = MetaData()


stations = Table(
'stations', meta,
Column('number', Integer, primary_key = True),
Column('name', String(128)),
Column('address', String(128)),
Column('pos_lat', Float),
Column('pos_lng', Float),
Column('bike_stands', Integer),
Column('available_bike_stands', Integer),
Column('available_bikes', Integer),
Column('last_update', DateTime)
)

#print(stations.number)

meta.create_all(engine)

In [ ]:
APIKEY = "faa0b3cb4ae90aba340567484b057e59c72b1a37"
NAME = "Dublin"
STATIONS = "https://api.jcdecaux.com/vls/v1/stations"


r = requests.get(STATIONS, params={"apiKey": APIKEY, "contract": NAME})
#x = JSON(json.loads(r.text))

# json = json.dumps(x)
# f = open("test.json", "w")
# f.write(json)
# f.close()
#
# print(x)


In [ ]:
JSON(r.json())

In [ ]:
def get_station(obj):
    return {'number': obj['number'],
            'name': obj['name'],
            'address': obj['address'],
            'pos_lng': obj['position']['lng'],
            'pos_lat': obj['position']['lat'],
            'bike_stands': obj['bike_stands'],
            'available_bike_stands': obj['available_bike_stands'],
            'available_bikes': obj['available_bikes'],
            'last_update': datetime.datetime.fromtimestamp(int(obj['last_update'] / 1e3))
           }

values = list(map(get_station, r.json()))
#print(values)
ins = stations.insert().values(values)
engine.execute(ins)